This is a python notebook made by **Yoonsoo P. Bach** to report the photometric results of the data obtained from the Seoul National University Astronomical Observatory (SNUO, also known as SAO, which I'd avoid due to the possible confusion with [Smithsonian Astrophysical Observatory](https://www.cfa.harvard.edu/sao)).

Observation was made on 2019-06-02 using the 1-m telescope at SNUO, using non-sidereal tracking mode for **(66391) 1999 KW4** for about 3.1 hours.

It is the third of four notebooks:

1. ``Reducer``
    - preprocessing, CR rejection, and WCS implementation.
2. ``Photometer``
    - ephemerides/PS1 query, centroid stars, Pillbox Aperture photometry, zeropoint fitting.
        * It's results are in the ``phot_targ.csv``, and summarized in ``main.pdf``.
3. ``latex_generator`` 
    - generates the report LaTeX file and you may compile it.
4. ``light_curve_alalyzer``
    - LC generator.

In [1]:
from pathlib import Path

BEGIN_STR_TWOCOL = r''' 
\title{2018-10-12/13 SNUO 1m Observation of (155140) 2005 UD}
\author{Yoonsoo P. Bach}
\date{\today}
\documentclass[10pt,twocolumn]{article}
\usepackage[a4paper, total={7.8in, 10.5in}]{geometry}
\usepackage{graphicx}
\usepackage{grffile}

\begin{document}
\maketitle 
\clearpage
'''

BEGIN_STR_ONECOL = r''' 
\title{2018-10-12/13 SNUO 1m Observation of (155140) 2005 UD}
\author{Yoonsoo P. Bach}
\date{\today}
\documentclass[10pt,onecolumn]{article}
\usepackage[a4paper, total={7.8in, 10.5in}]{geometry}
\usepackage{graphicx}
\usepackage{grffile}

\begin{document}
\maketitle 
\clearpage
'''


def fig_str_b(fpath):
    figstr = r'''
\begin{{figure}}[b!]
\centering
\includegraphics[width=\linewidth]{{{}}}
\end{{figure}}

    '''
    return figstr.format(FIGDIR_ / fpath.name)


def fig_str_h(fpath):
    figstr = r'''
\begin{{figure}}[h!]
\centering
\includegraphics[width=\linewidth]{{{}}}
\end{{figure}}

    '''
    return figstr.format(FIGDIR_ / fpath.name)


def path_parser(fpath):
    stem = '\_'.join(fpath.stem.split('_')[:2])
    splitted = fpath.stem.split('-')
    datetime = f"{splitted[4]}-{splitted[5]}"
    return stem, datetime


def iterate_allfigs(figpathlist):
    section_str = ("\n\n\clearpage\n" 
                   + r"\section{{ \texttt{{ {} }} }}" 
                   + "\n"
                   + r"\texttt{{ {} }}"
                   + "\n\n")

    stem_old, datetime_old = path_parser(figpathlist[0])
    latex_str = BEGIN_STR_TWOCOL + section_str.format(datetime_old, stem_old)

    for fpath in figpathlist:
        _str = latex_str
        stem_new, datetime_new = path_parser(fpath)

        if stem_new.startswith(stem_old):
            latex_str += fig_str_b(fpath)
        else:
            latex_str += "\n" + r"\clearpage \clearpage" + "\n"
            latex_str += section_str.format(datetime_new, stem_new)
            latex_str += fig_str_b(fpath)

        stem_old = stem_new
    latex_str += "\n\end{document}\n"
    return latex_str


def iterate(figpathlist):
    section_str = ("\n\n" 
                   + r"\section{{ \texttt{{ {} }} }}" 
                   + "\n"
                   + r"\texttt{{ {} }}"
                   + "\n\n")

    stem_old, datetime_old = path_parser(figpathlist[0])
    latex_str = BEGIN_STR_ONECOL + section_str.format(datetime_old, stem_old)

    for fpath in figpathlist:
        _str = latex_str
        stem_new, datetime_new = path_parser(fpath)

        if stem_new.startswith(stem_old):
            latex_str += fig_str_h(fpath)
        else:
            latex_str += "\n" + r"\clearpage \clearpage" + "\n"
            latex_str += section_str.format(datetime_new, stem_new)
            latex_str += fig_str_h(fpath)

        stem_old = stem_new
    latex_str += "\n\end{document}\n"
    return latex_str


    
REPORTDIR = Path("report")
FIGDIR = REPORTDIR / "figures"
FIGDIR_ = Path("figures")
figs = list(FIGDIR.glob("*.pdf"))
targs = list(FIGDIR.glob("*_target.pdf"))
zeros = list(FIGDIR.glob("*_zeropoint.pdf"))
targzp = targs + zeros
figs.sort()
targs.sort()
zeros.sort()
targzp.sort()

In [2]:
with open(REPORTDIR/"report.tex", "w+") as latexf:
    latexf.write(iterate_allfigs(figs))

# with open(REPORTDIR/"target_only.tex", "w+") as latexf:
#     latexf.write(iterate(targs))

# with open(REPORTDIR/"zeropint_only.tex", "w+") as latexf:
#     latexf.write(iterate(zeros))

with open(REPORTDIR/"target_and_zeropint_only.tex", "w+") as latexf:
    latexf.write(iterate(targzp))

In [3]:
%%bash
cd report
pdflatex report.tex
# pdflatex target_only.tex
# pdflatex zeropoint_only.tex
pdflatex target_and_zeropoint_only.tex
cd ..

bash: line 2: pdflatex: command not found
bash: line 5: pdflatex: command not found
